# 1. PubMed

Search PubMed for papers

https://www.ncbi.nlm.nih.gov/pubmed/

https://www.ncbi.nlm.nih.gov/books/NBK25499/

In [ ]:
import os
import lcp.reuse as reuse
from Bio import Entrez
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
PROJECT = 'eICU' # 'MIMIC'

def full_query(base_query, restriction_query):
    return ' AND '.join([base_query, restriction_query])

entrez_email = 'mimic-support@physionet.org'
if PROJECT == 'eICU':
    # Retrieves publications which cite this reference
    PMID = '30204154'

mimic_query = '(mimic-ii OR mimic-iii OR mimicii OR mimiciii OR mimic-2 OR mimic-3 OR mimic2 OR mimic3)'
if PROJECT == 'MIMIC':
    base_query = mimic_query
elif PROJECT == 'eICU':
    base_query = f'(eicu-crd OR "eICU Collaborative Research Database" OR (eicu AND ({mimic_query} OR database OR MIT OR Phillips)))'
# Other terms added to remove false positives. The more terms added without increasing FPs, the better.
if PROJECT == 'MIMIC':
    restriction_query = '(physionet OR icu OR "intensive care" OR "critical care")'
elif PROJECT == 'eICU':
    restriction_query = '("intensive care" OR "critical care")'
# more restriction keyword ideas: clinical, database, waveform (not suitable on their own due to general mimic term)
full_query = full_query(base_query, restriction_query)

if PMID:
    search_strings = [
        'PMID'
    ]
else:
    search_strings = [
        base_query#,
        # full_query
    ]

In [ ]:
# search_results = reuse.search_list(search_strings, entrez_email)
if PMID:
    citations = []
    pmc_ids = Entrez.read(Entrez.elink(dbfrom='pubmed', db='pmc', LinkName='pubmed_pmc_refs', from_uid=PMID))
    pmc_ids = [list(i.values())[0] for i in pmc_ids[0]['LinkSetDb'][0]['Link']]
    pm_ids = Entrez.read(Entrez.elink(dbfrom='pmc', db='pubmed', LinkName='pmc_pubmed', from_uid=','.join(pmc_ids)))
    for pm_id in [list(i.values())[0] for i in pm_ids[0]['LinkSetDb'][0]['Link']]:
        citations.append(pm_id)
    search_results_all = reuse.search_list(entrez_email, ids=citations, all=True)
else:
    search_results_all = reuse.search_list(entrez_email, search_strings=search_strings, all=True)

In [ ]:
# Display number of results
if PMID:
    result = search_results_all['PMID']
    print(f'PMID:\n - Count: {len(result.index)}')
else:
    for ss in search_strings:
        result = search_results_all[ss]
        print(f'{ss}:\n - Count: {len(result.index)}')

In [ ]:
# Write the titles to files
write_dir = os.path.join('search_results', PROJECT, 'pubmed')
os.makedirs(write_dir, exist_ok=True)

if PMID:
    search_results_all['PMID'].to_csv(os.path.join(write_dir, 'PMID-citations-all.csv'), index=False)
else:
    search_results_all[base_query].to_csv(os.path.join(write_dir, 'without-constraints-all.csv'), index=False)
    search_results_all[full_query].to_csv(os.path.join(write_dir, 'with-constraints-all.csv'), index=False)

# base_query_file = os.path.join(write_dir, 'without-constraints.txt')
# with open(base_query_file, 'w') as f:
#     for line in search_results[base_query].paper_titles:
#         f.write(line+'\n')

# full_query_file = os.path.join(write_dir, 'with-constraints.txt')
# with open(full_query_file, 'w') as f:
#     for line in search_results[full_query].paper_titles:
#         f.write(line+'\n')

In [ ]:
# Create a new one for each year
if PMID:
    all_years = sorted(set(list(search_results_all['PMID']['Publication Year'])))
    for year in all_years:
        year_df = search_results_all['PMID'][search_results_all['PMID']['Publication Year'] == year]
        year_df.to_csv(os.path.join(write_dir, f'PMID-citations-all_{year}.csv'), index=False)
else:
    all_years = sorted(set(list(search_results_all[full_query]['Publication Year'])))
    for year in all_years:
        year_df_without = search_results_all[base_query][search_results_all[base_query]['Publication Year'] == year]
        year_df_with = search_results_all[full_query][search_results_all[full_query]['Publication Year'] == year]
        year_df_without.to_csv(os.path.join(write_dir, f'without-constraints-all_{year}.csv'), index=False)
        year_df_with.to_csv(os.path.join(write_dir, f'with-constraints-all_{year}.csv'), index=False)

In [ ]:
# Create a histogram of number of publications each year
if PMID:
    all_years = sorted(set(list(search_results_all['PMID']['Publication Year'])))

    years = []
    for year in all_years:
        year_df = search_results_all['PMID'][search_results_all['PMID']['Publication Year'] == year]

        for _ in range(len(year_df.index)):
            years.append(year)

    plt.figure(figsize=(10,6))
    plt.hist(years, facecolor='k', edgecolor='w', bins=np.arange(len(all_years)+1)-0.5)
    plt.savefig(os.path.join(write_dir, 'PMID-citations_histogram.jpg'))
    plt.xlabel('Year', fontsize=12)
    plt.ylabel('Number of Publications', fontsize=12)
    plt.show()
else:
    all_years = sorted(set(list(search_results_all[full_query]['Publication Year'])))

    year_without = []
    year_with = []
    for year in all_years:
        year_df_without = search_results_all[base_query][search_results_all[base_query]['Publication Year'] == year]
        year_df_with = search_results_all[full_query][search_results_all[full_query]['Publication Year'] == year]

        for _ in range(len(year_df_without.index)):
            year_without.append(year)
        for _ in range(len(year_df_with.index)):
            year_with.append(year)

    plt.figure(figsize=(10,6))
    plt.hist(year_without, facecolor='k', edgecolor='w', bins=np.arange(len(all_years)+1)-0.5)
    plt.savefig(os.path.join(write_dir, 'without-constraints_histogram.jpg'))
    plt.xlabel('Year', fontsize=12)
    plt.ylabel('Number of Publications', fontsize=12)
    plt.show()

    plt.figure(figsize=(10,6))
    plt.hist(year_with, facecolor='k', edgecolor='w', bins=np.arange(len(all_years)+1)-0.5)
    plt.savefig(os.path.join(write_dir, 'with-constraints_histogram.jpg'))
    plt.xlabel('Year', fontsize=12)
    plt.ylabel('Number of Publications', fontsize=12)
    plt.show()

In [ ]:
# DO: Create a file called without-constraints-inspected.tsv and mark the second column with T/F for true/false positives.


In [ ]:
# Look at the effect of restricting the search by additional criteria.
# The differences show that many false positives, and a few true positives, are removed.
reuse.showdiff(search_results[search_strings[0]],
               search_results[search_strings[1]])

In [ ]:
# Read in the labelled results for the general unconstrained search query
labelled_results = pd.read_csv(os.path.join(write_dir, 'without-constraints-inspected.tsv'), delimiter='\t', header=None)
false_positives = labelled_results.loc[labelled_results[1]=='F'][0].values
true_positives = labelled_results.loc[labelled_results[1]=='T'][0].values
print('Number of results found using the unconstrained search term:', len(labelled_results))
print('Number of false positives:',len(false_positives))
print('Number of true positives:',len(true_positives))

constrained_titles = search_results[search_strings[1]].paper_titles
print('\nCompare ^ true positives with:')
print('Number of results from the constrained search term:', len(constrained_titles))

missed_papers = set(true_positives) - set(constrained_titles)
print('Number of missed true positives:', len(missed_papers))

In [ ]:
# Take a look at some True positives missed by the constrained search term to figure out what else you can add.
# Write to a file to label comments.
write_dir = os.path.join('search_results', 'pubmed')

missed_papers_file = os.path.join(write_dir, 'missed-papers.tsv')

with open(missed_papers_file, 'w') as f:
    for line in missed_papers:
        f.write(line+'\n')

display(missed_papers)

# 2. Web of Science

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import time
from IPython.display import display


base_mimic_query = '(mimic-ii OR mimic-iii OR mimicii OR mimiciii OR mimic-2 OR mimic-3 OR mimic2 OR mimic3)'
restriction_query = '(physionet OR icu OR “intensive care” OR “critical care”)'
def full_query(base_query, restriction_query):
    return ' AND '.join([base_query, restriction_query])
full_mimic_query = full_query(base_mimic_query, restriction_query)
#base_search_url = 'https://apps.webofknowledge.com/WOS_GeneralSearch_input.do?product=WOS&search_mode=GeneralSearch&SID=2F46AeWkMQBRAZlzDWm&preferencesSaved='
base_search_url = 'https://apps.webofknowledge.com/WOS_GeneralSearch_input.do?product=WOS&search_mode=GeneralSearch&SID=1AnC2UMojuKrtrl7T5R&preferencesSaved='

all_titles = []

In [ ]:
# Get to the search page
driver = webdriver.Firefox()
driver.get(base_search_url)

# Input the query string
time.sleep(2.5)
searchbox = driver.find_element_by_id('value(input1)')
searchbox.send_keys(full_mimic_query)

# Search
time.sleep(1)
searchbutton = driver.find_element_by_css_selector('.standard-button.primary-button.large-search-button')
searchbutton.click()

# Get the total number of pages
npages = int(driver.find_element_by_id('pageCount.top').text)

In [ ]:
# Get the titles!!!
while True:
    # Get the current page number
    pagenum = int(driver.find_element_by_class_name('goToPageNumber-input').get_property('value'))

    # Get the titles. This also captures the journals. So every second value is not a title.
    elements = driver.find_elements_by_class_name('smallV110')

    for e in elements[::2]:
        all_titles.append(e.find_element_by_tag_name('value').text)
        
    if pagenum < npages:
        nextbutton = driver.find_element_by_class_name('paginationNext')
        nextbutton.click()
    else:
        print('Got all paper titles!')
        driver.close()
        break
        
all_titles = set(all_titles)
#all_titles.remove('')
all_titles = [t.lower() for t in list(all_titles)]

In [ ]:
display(all_titles)

In [ ]:
# Write the titles to files
write_dir = os.path.join('search_results/wos')

full_query_file = os.path.join(write_dir, 'with-constraints.txt')

with open(full_query_file, 'w') as f:
    for line in all_titles:
        f.write(line+'\n')

# 3. SCOPUS

Shit search

# 4. IEEE

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import time
from IPython.display import display
import os

base_mimic_query = '(mimic-ii OR mimic-iii OR mimicii OR mimiciii OR mimic-2 OR mimic-3 OR mimic2 OR mimic3)'
restriction_query = '(physionet OR icu OR “intensive care” OR “critical care”)'
def full_query(base_query, restriction_query):
    return ' AND '.join([base_query, restriction_query])
full_mimic_query = full_query(base_mimic_query, restriction_query)
base_search_url = 'http://ieeexplore.ieee.org/search/advsearch.jsp?expression-builder'
all_titles = []

In [ ]:
# Get to the search page
driver = webdriver.Firefox()
driver.get(base_search_url)

# Input the query string
searchbox = driver.find_element_by_id('expression-textarea')
searchbox.send_keys(full_mimic_query)
# Select the 'full text and metadata' box
radiobutton = driver.find_element_by_id('Search_All_Text')
radiobutton.click()

# Search
time.sleep(1)
searchbutton = driver.find_element_by_class_name('stats-Adv_Command_search')
searchbutton.click()

# Get the total number of pages
#npages = int(driver.find_element_by_id('pageCount.top').text)

In [ ]:
# Get the titles!!!
while True:
    # let the page load
    time.sleep(2)
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(0.5)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        
    # Get the titles.
    # They are in: <h2 class="result-item-title"><a class="ng-binding ng-scope">title</a></h2>
    elements = driver.find_elements_by_class_name('result-item-title')
    for e in elements:
        # Text may appear with "[::sometext::]"
        all_titles.append(e.find_element_by_tag_name('a').get_attribute('text').replace('[::', '').replace('::]', ''))
        # New line separated journal info and such
        #all_titles.append(e.text.split('\n')[0])
    
    # Click next page if any
    
    e = driver.find_element_by_class_name('next')
    if 'disabled' in e.get_attribute('class'):
        print('Got all paper titles!')
        driver.close()
        break
    else:
        nextbutton = driver.find_element_by_link_text('>')
        nextbutton.click()

all_titles = set(all_titles)
all_titles = [t.lower() for t in list(all_titles)]

In [ ]:
print(len(all_titles))
display(all_titles)

In [ ]:
# Write the titles to files
write_dir = os.path.join('search_results/ieee')

full_query_file = os.path.join(write_dir, 'with-constraints.txt')

with open(full_query_file, 'w') as f:
    for line in all_titles:
        f.write(line+'\n')

# Combining Results - pubmed, wos, ieee

In [ ]:
result_dir = 'search_results'

combined_results = []
for service in ['pubmed', 'wos', 'ieee']:
    # For pubmed, get the curated true positives from the unconstrained search instead
    if service == 'pubmed':
        df = pd.read_csv(os.path.join(result_dir, service, 'without-constraints-inspected.tsv'), delimiter='\t', header=None)
        service_results = list(df.loc[df[1]=='T'][0].values)
    # For other services, get the constrained search results
    else:
        with open(os.path.join(result_dir, service, 'with-constraints.txt')) as f:
            service_results = f.readlines()
    print('Number of results from service '+service+': '+str(len(service_results)))
    combined_results = combined_results + [r.strip() for r in service_results]

print('\nTotal number of non-unique results: ', len(combined_results))
combined_results = sorted(list(set(combined_results)))
print('Total number of unique results: ', len(combined_results))
with open(os.path.join(result_dir, 'combined', 'with-constraints.txt'), 'w') as f:
    for r in combined_results:
        f.write(r+'\n')

In [ ]:
# may 21 2018
Number of results from service pubmed: 155
Number of results from service wos: 152
Number of results from service ieee: 322

Total number of non-unique results:  629
Total number of unique results:  456


# Attempting to parse GS automatically failed. Below is evidence of failure. Can ignore...

# N. Search Google Scholar

Packages found online:
- https://github.com/ckreibich/scholar.py
- https://github.com/venthur/gscholar
- https://github.com/adeel/google-scholar-scraper
- http://code.activestate.com/recipes/523047-search-google-scholar/
- https://github.com/erdiaker/torrequest
- https://github.com/NikolaiT/GoogleScraper


- https://stackoverflow.com/questions/8049520/web-scraping-javascript-page-with-python


Query: `("mimic ii" OR "mimic iii") AND ("database" OR "clinical" OR "waveform" OR ICU)`

https://scholar.google.com/scholar?q=%28mimic-ii+OR+mimic-iii%29&btnG=&hl=en&as_sdt=1%2C22&as_vis=1

https://scholar.google.com/scholar/help.html


https://superuser.com/questions/565722/how-to-config-tor-to-use-a-http-socks-proxy

## Requirements

1. Browse with JS enabled. requests library uses http. Otherwise google will think (correctly) that you are a robot.
2. Change IP every time, or google will block.

In [ ]:
from scholarly import scholarly

In [ ]:
PROJECT = 'eICU'

# Search by Google Scholar publication ID
search_results = []
if PROJECT == 'eICU':
    try:
        search_results.append(scholarly.search_citedby(11878669525996073977))
    except Exception as e:
        print(e)

total_results = search_results[0].total_results

In [ ]:
# all_pubs = list(search_results)
all_pubs = []
for i in range(total_results):
    print(100*i/total_results)
    try:
        ct = next(search_results)
        all_pubs.append({
            'title': ct['bib']['title'],
            'pub_year': ct['bib']['pub_year']
        })
    except:
        pass
# print(all_pubs)